In [4]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
itxt="5.txt"
with open(itxt,"r") as f:
    datalist = f.readlines()
datanp=np.zeros((len(datalist),11))
itxt1="7.txt"
with open(itxt1,"r") as f:
    datalist1 = f.readlines()
datanp1=np.zeros((len(datalist1),11))

count=0
for i in datalist:
    datai=i.split("\n")[0].split("\t")
    #print(datai)
    for j in range(11):
        #print(j)
        datanp[count:count+1,j]=float(datai[j])
    count=count+1
print(datanp[0:3,:],datanp.shape)
count1=0
for i in datalist1:
    datai=i.split("\n")[0].split("\t")
    #print(datai)
    for j in range(11):
        #print(j)
        datanp1[count1:count1+1,j]=float(datai[j])
    count1=count1+1
print(datanp1[0:3,:],datanp1.shape)

[[1670.05     0.       0.       0.       0.      68.585    0.       0.
    36.36   -11.32   -88.09 ]
 [1670.06     0.       0.       0.       0.      68.585    0.       0.
    36.36   -11.32   -88.08 ]
 [1670.07     0.       0.       0.       0.      68.585    0.       0.
    36.36   -11.32   -88.07 ]] (19997, 11)
[[880.11         0.           0.           0.           0.
   67.8328125    0.           0.          36.51       -11.24690476
  -89.22      ]
 [880.12         0.           0.           0.           0.
   67.8328125    0.           0.          36.51       -11.24928571
  -89.22      ]
 [880.13         0.           0.           0.           0.
   67.8328125    0.           0.          36.51       -11.25
  -89.22      ]] (15991, 11)


In [3]:
train_data = datanp
test_data = datanp1
def get_XY(dat, time_steps):
    # Indices of target array
    Y_ind = np.arange(time_steps, dat.shape[0], time_steps)
    Y = dat[Y_ind,:]
    # Prepare X
    rows_x = Y.shape[0]
    X = dat[range(time_steps*rows_x),:]
    X = np.reshape(X, (rows_x, time_steps, 11))    
    return X, Y
time_steps = 10
trainX, trainY = get_XY(train_data, time_steps)
testX, testY = get_XY(test_data, time_steps)
print(trainX.shape,trainY.shape)
print(testX.shape,testY.shape)

(1999, 10, 11) (1999, 11)
(1599, 10, 11) (1599, 11)


In [5]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [10]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)
    return keras.Model(inputs, outputs)

In [7]:
x_train=trainX[:,:,1:5]
y_train=trainY[:,5:6]
x_test=testX[:,:,1:5]
y_test=testY[:,5:6]

In [12]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    #callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 10, 4)]      0           []                               
                                                                                                  
 layer_normalization_24 (LayerN  (None, 10, 4)       8           ['input_4[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 multi_head_attention_12 (Multi  (None, 10, 4)       19460       ['layer_normalization_24[0][0]', 
 HeadAttention)                                                   'layer_normalization_24[0][0]'] 
                                                                                            

283.1919250488281